In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "mistralai/Mistral-7B-Instruct-v0.1"

# Load tokenizer and model (uses your logged-in credentials)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,   # GPU optimized for 6GB VRAM
    device_map="auto"
)

print("✅ Model loaded on:", next(model.parameters()).device)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Model loaded on: cuda:0


In [ ]:
import pandas as pd
from tqdm import tqdm
import re
import time

# Your current function (no prints)
def classify_stereotype_with_mistral_v5_silent(line):
   prompt = f"""
   You are a language model trained to detect gender stereotypes in lines from movie scripts.
   Your task is to classify each line into exactly one of the following 14 categories, based on the presence of stereotypical portrayals—especially of women.
   Assign only **one** label per line. Be careful to read subtle biases. Return only the **label** (not the line, not any explanation).
   Here are the 14 valid categories:
    1. occupation_gap → The character lacks a job, career mention, or professional identity.
      - Ex: "She stayed home while the others went to work."

    2. agency_gap → The character lacks independence or decision-making ability; is passive or submissive.
      - Ex: "She looked at him, waiting for his decision."

    3. appearance_focus → Emphasis on physical appearance, beauty, clothing, or body—but not sexual objectification.
      - Ex: "She walked in wearing a tight red dress that hugged every curve."

    4. relationship_only → Introduced only as a relation to another (wife, daughter, etc.).
      - Ex: "That's John's wife, the one sitting by the window."

    5. screen_time_disparity→ Line shows that women have very limited screen/dialog presence.
      - Ex: "The male characters dominated every major scene, while she had only two lines."

    6. dialogue_initiation_gap → The woman doesn't start conversations or only speaks when spoken to.
      - Ex: "She rarely spoke unless someone asked her a question first."

    7. emotional_typecast → The woman is only shown as overly emotional, irrational, or tearful.
      - Ex: "She burst into tears again, as usual."

    8. domesticity_emphasis → Emphasis on domestic roles (cooking, cleaning, homemaking).
      - Ex: "She was happiest baking cookies in her cozy kitchen."

    9. objectification → The body is sexualized visually or narratively. Often includes gaze, zoom-ins, or body parts.
      - Ex: "The camera panned slowly from her legs to her chest."

    10. victim_only → The woman appears only to suffer harm or be rescued, with no further agency.
      - Ex: "She screamed for help as the villain grabbed her."

    11. intelligence_undermined → The woman is portrayed as dumb, ditzy, or confused—especially in academic/intellectual areas.
      - Ex: "She looked confused by the math problem and giggled nervously."

    12. support_role_only → Exists only to encourage, cheer for, or help a man without her own arc.
      - Ex: "She cheered him on from the sidelines as he accepted his award."

    13. villainization → Woman is unfairly portrayed as evil, manipulative, or cruel (e.g. stepmother, seductress).
      - Ex: "The evil stepmother glared with contempt and plotted her revenge."

    14. none → The line is empowering or neutral. No stereotype or bias.
      - Ex: "She led the mission with confidence and precision."

    ---

    Read the line carefully. Choose the **most accurate single category** from the above list.
    If the line contains no bias, use `none`.

    Line: "{line}"
    Label:"""
   inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
   outputs = model.generate(
       **inputs,
       max_new_tokens=10,
       pad_token_id=tokenizer.eos_token_id,
   )
   decoded = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
   label_matches = re.findall(r"Label:\s*(\w+)", decoded)
   valid_labels = {
     "occupation_gap",
     "agency_gap",
     "appearance_focus",
     "relationship_only",
     "screen_time_disparity",
     "dialogue_initiation_gap",
     "emotional_typecast",
     "domesticity_emphasis",
     "objectification",
     "victim_only",
     "intelligence_undermined",
     "support_role_only",
     "villainization",
     "none"
   }
   label = next((lbl.lower() for lbl in reversed(label_matches) if lbl.lower() in valid_labels), "none")
   return label

In [ ]:
test_sentences = {
    "occupation_gap": "She stayed home while the others went to work.",
    "agency_gap": "She looked at him, waiting for his decision.",
    "appearance_focus": "She walked in wearing a tight red dress that hugged every curve.",
    "relationship_only": "That's John's wife, the one sitting by the window.",
    "screen_time_disparity": "The male characters dominated every major scene, while she had only two lines.",
    "dialogue_initiation_gap": "She rarely spoke unless someone asked her a question first.",
    "emotional_typecast": "She burst into tears again, as usual.",
    "domesticity_emphasis": "She was happiest baking cookies in her cozy kitchen.",
    "objectification": "The camera panned slowly from her legs to her chest.",
    "victim_only": "She screamed for help as the villain grabbed her.",
    "intelligence_undermined": "She looked confused by the math problem and giggled nervously.",
    "support_role_only": "She cheered him on from the sidelines as he accepted his award.",
    "villainization": "The evil stepmother glared with contempt and plotted her revenge.",
    "none": "She led the mission with confidence and precision."
}

for expected, sentence in test_sentences.items():
    predicted = classify_stereotype_with_mistral_v5_silent(sentence)
    print(f"Expected: {expected} | Predicted: {predicted} | Line: {sentence}")

Expected: occupation_gap | Predicted: occupation_gap | Line: She stayed home while the others went to work.
Expected: agency_gap | Predicted: agency_gap | Line: She looked at him, waiting for his decision.
Expected: appearance_focus | Predicted: appearance_focus | Line: She walked in wearing a tight red dress that hugged every curve.
Expected: relationship_only | Predicted: relationship_only | Line: That's John's wife, the one sitting by the window.
Expected: screen_time_disparity | Predicted: screen_time_disparity | Line: The male characters dominated every major scene, while she had only two lines.
Expected: dialogue_initiation_gap | Predicted: dialogue_initiation_gap | Line: She rarely spoke unless someone asked her a question first.
Expected: emotional_typecast | Predicted: emotional_typecast | Line: She burst into tears again, as usual.
Expected: domesticity_emphasis | Predicted: domesticity_emphasis | Line: She was happiest baking cookies in her cozy kitchen.
Expected: objectific

In [ ]:
# Load your full CSV
df = pd.read_csv("cleaned_script_data_plots.csv")
chunk_size = 250  # Experiment with 200–300 per chunk

for start in range(0, len(df), chunk_size):
    end = min(start + chunk_size, len(df))
    chunk = df.iloc[start:end].copy()

    print(f"🧠 Processing lines {start} to {end}")
    tqdm.pandas()
    chunk["stereotype_type"] = chunk["line"].progress_apply(classify_stereotype_with_mistral_v5_silent)

    chunk.to_csv(f"chunk_{start}_{end}.csv", index=False)
    print(f"✅ Saved: chunk_{start}_{end}.csv")

    # Free GPU memory
    torch.cuda.empty_cache()
    time.sleep(2)

🧠 Processing lines 0 to 250


100%|██████████| 250/250 [08:52<00:00,  2.13s/it]


✅ Saved: chunk_0_250.csv
🧠 Processing lines 250 to 500


100%|██████████| 250/250 [08:45<00:00,  2.10s/it]


✅ Saved: chunk_250_500.csv
🧠 Processing lines 500 to 750


...

100%|██████████| 250/250 [08:43<00:00,  2.09s/it]


✅ Saved: chunk_1000_1250.csv
🧠 Processing lines 1250 to 1500


100%|██████████| 250/250 [08:40<00:00,  2.08s/it]


✅ Saved: chunk_1250_1500.csv
🧠 Processing lines 1500 to 1750


 65%|██████▌   | 163/250 [05:32<03:17,  2.27s/it]

In [ ]:
import glob
import pandas as pd
files = sorted(glob.glob("chunk_*.csv"))
df_all = pd.concat([pd.read_csv(f) for f in files], ignore_index=True)
df_all.to_csv("ai_stereotype_annotated_final.csv", index=False)
print("✅ All chunks merged and saved.")

✅ All chunks merged and saved.
